In [15]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from pylab import rcParams
rcParams['figure.figsize'] = 20, 8
import warnings
warnings.simplefilter('ignore')
from sklearn.svm import LinearSVC
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from pylab import rcParams
rcParams['figure.figsize'] = 20, 8
import warnings
warnings.simplefilter('ignore')
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
df = pd.read_csv('data_full.csv', header = 0, sep = ',')

 # Переведем категориальную переменную в числа:
df['category_id'] = df['category'].factorize()[0]
df1 =pd.Index(df["category"]).value_counts()
df_category = pd.DataFrame(df1)

def predictmodel(question):
     # добавляем новую строку с запросом
    df_temp = {'category': None, 'text':question, 'category_id': None}
    numberOfRows = len(df)+1
    df_for_question_full = df
    df_for_question = pd.DataFrame(index=np.arange(numberOfRows, numberOfRows+1), data = df_temp, columns=('category','executor','id','text','theme','theme_id','category_id','executor_id') )
    df_for_question_full = pd.concat([df_for_question_full, df_for_question])
    # Векторизация текста

    stopWords = stopwords.words('russian')
    tfidf = TfidfVectorizer(sublinear_tf=True, min_df=3, norm='l2', encoding='utf-8', ngram_range=(1, 2), stop_words=stopWords)
    features = tfidf.fit_transform(df_for_question_full.text).toarray()
    labels = df.category_id
    features_for_predict = features[numberOfRows-1:numberOfRows,::]
    features_for_model = features[0:numberOfRows-1,::] #Все строки кроме последней
    category_id_df = df[['category', 'category_id']].drop_duplicates().sort_values('category_id')
    #Обучение модели и предсказание
    model = LinearSVC()
    X_train = features_for_model
    y_train = labels
    model.fit(X_train, y_train)
    answer = model.predict(features_for_predict)[0]
    return answer
  

In [ ]:
from werkzeug.wrappers import Request, Response
import pprint

pp = pprint.PrettyPrinter(indent=4)

@Request.application
def application(request):
    #print request
    q = request.get_data(False, True, False)
    pp.pprint(q)
    a = predictmodel(q)
    return Response(str(a))

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('192.168.137.146', 4000, application)

 * Running on http://192.168.137.146:4000/ (Press CTRL+C to quit)
192.168.137.1 - - [07/Jul/2019 15:16:51] "POST / HTTP/1.1" 200 -


u'\ufffd\ufffd\ufffd \ufffd\ufffd\ufffd\ufffd \ufffd\ufffd\ufffd\ufffd\ufffd\ufffd \ufffd \ufffd\ufffd\ufffd?'


192.168.137.1 - - [07/Jul/2019 15:18:27] "GET /favicon.ico HTTP/1.1" 200 -


u''


192.168.137.1 - - [07/Jul/2019 15:21:04] "POST / HTTP/1.1" 200 -


u'\u043d\u0435\u0442 \u0432\u043e\u0434\u044b \u0441\u043e\u0432\u0441\u0435\u043c \u0443 \u043d\u0430\u0441?'


In [ ]:
#question = "Подскажите по отключению горячей воды"
#question = "Тульская 6А - 13. В мае 18 года начался ремонт балконов. Подошла к ГЛАВНОМУ- сверху крошится потолок. Все лето слышала ответ, чтобы попасть на застекленный мой балкон, надо снять окна и заплатить. Я не согласилась. Успокоилась, сделала ремонт. 4.02.19 пришли по поводу ремонта балкона. Почему мне все лето отказывали в ремонте, хотя я все лето просила их, почему летом никто не делал внутри балкон?"
#question = "У остановочного павильона повреждена крыша"
#question = "У нас нет воды! Никакой!"
#question = "Образовалась выбоина, яма на на тротуаре Не удовлетворительное состояние тротуаров: тротуарная плитка местами отсутствует; местами сломана и просто болтается!"
#predictmodel(question)

0	Нарушение правил содержания стройплощадок

1	Капитальный ремонт многоквартирного дома

2	Автомобиль припаркован на газоне

3	Остановки общественного транспорта

4	Благоустройство дворовой территории

5	Парки и скверы

6	Текущее содержание дворовой территории

7	Текущее содержание дорог и ремонт

8	Отсутствует водоснабжение



